In [74]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import sqlite3

In [75]:
# Connect to the SQLite database
conn = sqlite3.connect('db/imdb.db')

In [76]:
# Load the datasets from the SQLite database
names_df = pd.read_sql_query('SELECT nconst, primaryName FROM Names', conn)

In [77]:
names_df

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
12571732,nm9993714,Romeo del Rosario
12571733,nm9993716,Essias Loberg
12571734,nm9993717,Harikrishnan Rajan
12571735,nm9993718,Aayush Nair


In [78]:
principals_df = pd.read_sql_query('SELECT tconst, nconst, category FROM Principals WHERE category IN ("director", "actor", "actress")', conn)

In [79]:
principals_df

,tconst,nconst,category
0,tt0000001,nm0005690,director
1,tt0000002,nm0721526,director
2,tt0000003,nm0721526,director
3,tt0000004,nm0721526,director
4,tt0000005,nm0443482,actor
...,...,...,...
28595706,tt9916880,nm0254176,actress
28595707,tt9916880,nm0286175,actor
28595708,tt9916880,nm10535738,actress
28595709,tt9916880,nm0996406,director


In [80]:
movies_df = pd.read_sql_query('SELECT Titles.tconst, primaryTitle, genres FROM Titles INNER JOIN Ratings ON Titles.tconst = Ratings.tconst WHERE\
                              titleType = "movie" ORDER BY Ratings.averageRating DESC, Titles.startYear DESC', conn)

In [81]:
movies_df

,tconst,primaryTitle,genres
0,tt12304768,Neon Bleed,"Crime,Drama,Music"
1,tt12654846,Rudy,Drama
2,tt15299916,Jaishree Krishh,"Drama,Romance"
3,tt15975334,Mirror Moves,Thriller
4,tt19370518,Decent Reflection,\N
...,...,...,...
292597,tt0444711,Der Würger der Welt,"Adventure,Crime,Mystery"
292598,tt7332864,Der Schuldschein des Pandola,"Adventure,Crime"
292599,tt0427653,Perils of the Secret Service,"Adventure,Thriller"
292600,tt0005411,Graustark,Adventure


In [82]:
merged_df = pd.merge(pd.merge(names_df, principals_df), movies_df, on="tconst")

In [83]:
merged_df

,nconst,primaryName,tconst,category,primaryTitle,genres
0,nm0000001,Fred Astaire,tt0025164,actor,The Gay Divorcee,"Comedy,Musical,Romance"
1,nm0001677,Ginger Rogers,tt0025164,actress,The Gay Divorcee,"Comedy,Musical,Romance"
2,nm0002143,Edward Everett Horton,tt0025164,actor,The Gay Divorcee,"Comedy,Musical,Romance"
3,nm0103567,Alice Brady,tt0025164,actress,The Gay Divorcee,"Comedy,Musical,Romance"
4,nm0762263,Mark Sandrich,tt0025164,director,The Gay Divorcee,"Comedy,Musical,Romance"
...,...,...,...,...,...,...
1409102,nm9986784,Shinkyu Choi,tt10925772,director,Hello Carbot The Movie: Save The Moon,"Adventure,Animation,Family"
1409103,nm9987477,Eric Overton,tt8944962,director,Collodion: The Process of Preservation,Documentary
1409104,nm9992852,Dahl Lee,tt8757056,director,Tobot Movie: Attack of Robot Force,"Action,Adventure,Animation"
1409105,nm9992853,Dongwoo Ko,tt8757056,director,Tobot Movie: Attack of Robot Force,"Action,Adventure,Animation"


In [84]:
aggregated_df = merged_df.groupby('tconst').agg({
    'primaryTitle': 'first',
    'genres': lambda x: ', '.join(x.dropna()) if x.notna().any() else 'Unknown',
    'nconst': lambda x: ', '.join(x),
    'category': lambda x: ', '.join(x),
     'primaryName': lambda x: ', '.join(x)
}).reset_index()


In [85]:
aggregated_df

,tconst,primaryTitle,genres,nconst,category,primaryName
0,tt0000009,Miss Jerry,"Romance, Romance, Romance, Romance","nm0063086, nm0085156, nm0183823, nm1309758","actress, director, actor, actor","Blanche Bayliss, Alexander Black, William Cour..."
1,tt0000147,The Corbett-Fitzsimmons Fight,"Documentary,News,Sport",nm0714557,director,Enoch J. Rector
2,tt0000502,Bohemios,"\N, \N, \N","nm0063413, nm0215752, nm0252720","director, actor, actor","Ricardo de Baños, Antonio del Pozo, El Mochuelo"
3,tt0000574,The Story of the Kelly Gang,"Action,Adventure,Biography, Action,Adventure,B...","nm0846879, nm0846887, nm0846894, nm1431224, nm...","director, actress, actor, actor, actor","Charles Tait, Elizabeth Tait, John Tait, Nicho..."
4,tt0000591,The Prodigal Son,"Drama, Drama, Drama, Drama, Drama","nm0141150, nm0332182, nm0906197, nm1323543, nm...","director, actor, actor, actress, actress","Michel Carré, Henri Gouget, Georges Wague, Chr..."
...,...,...,...,...,...,...
289027,tt9916270,Il talento del calabrone,"Thriller, Thriller, Thriller, Thriller, Thriller","nm0144812, nm1480867, nm2063290, nm3080119, nm...","actor, director, actress, actor, actress","Sergio Castellitto, Giacomo Cimini, Anna Fogli..."
289028,tt9916362,Coven,"Drama,History, Drama,History, Drama,History, D...","nm0107165, nm0266723, nm10678594, nm1893148, n...","actor, actor, actress, director, actress","Alex Brendemühl, Daniel Fanego, Garazi Urkola,..."
289029,tt9916428,The Secret of China,"Adventure,History,War, Adventure,History,War, ...","nm0422639, nm0910951, nm3370295, nm3611859, nm...","actress, director, actor, actor, actor, actor,...","Wenli Jiang, Jixing Wang, Shenyang Xiao, Kenan..."
289030,tt9916538,Kuambil Lagi Hatiku,"Drama, Drama, Drama, Drama, Drama","nm10041459, nm1266058, nm1417182, nm4457074, n...","actor, actress, actress, director, actress","Sahil Shah, Ria Irawan, Cut Mini Theo, Azhar K..."


In [86]:
aggregated_df['features'] = aggregated_df['primaryTitle'] + ' ' + \
                             aggregated_df['genres'] + ' ' + \
                             aggregated_df['nconst'] + ' ' + \
                            aggregated_df['category'] + ' ' + \
                             aggregated_df['primaryName']

In [87]:
aggregated_df

,tconst,primaryTitle,genres,nconst,category,primaryName,features
0,tt0000009,Miss Jerry,"Romance, Romance, Romance, Romance","nm0063086, nm0085156, nm0183823, nm1309758","actress, director, actor, actor","Blanche Bayliss, Alexander Black, William Cour...","Miss Jerry Romance, Romance, Romance, Romance ..."
1,tt0000147,The Corbett-Fitzsimmons Fight,"Documentary,News,Sport",nm0714557,director,Enoch J. Rector,"The Corbett-Fitzsimmons Fight Documentary,News..."
2,tt0000502,Bohemios,"\N, \N, \N","nm0063413, nm0215752, nm0252720","director, actor, actor","Ricardo de Baños, Antonio del Pozo, El Mochuelo","Bohemios \N, \N, \N nm0063413, nm0215752, nm02..."
3,tt0000574,The Story of the Kelly Gang,"Action,Adventure,Biography, Action,Adventure,B...","nm0846879, nm0846887, nm0846894, nm1431224, nm...","director, actress, actor, actor, actor","Charles Tait, Elizabeth Tait, John Tait, Nicho...","The Story of the Kelly Gang Action,Adventure,B..."
4,tt0000591,The Prodigal Son,"Drama, Drama, Drama, Drama, Drama","nm0141150, nm0332182, nm0906197, nm1323543, nm...","director, actor, actor, actress, actress","Michel Carré, Henri Gouget, Georges Wague, Chr...","The Prodigal Son Drama, Drama, Drama, Drama, D..."
...,...,...,...,...,...,...,...
289027,tt9916270,Il talento del calabrone,"Thriller, Thriller, Thriller, Thriller, Thriller","nm0144812, nm1480867, nm2063290, nm3080119, nm...","actor, director, actress, actor, actress","Sergio Castellitto, Giacomo Cimini, Anna Fogli...","Il talento del calabrone Thriller, Thriller, T..."
289028,tt9916362,Coven,"Drama,History, Drama,History, Drama,History, D...","nm0107165, nm0266723, nm10678594, nm1893148, n...","actor, actor, actress, director, actress","Alex Brendemühl, Daniel Fanego, Garazi Urkola,...","Coven Drama,History, Drama,History, Drama,Hist..."
289029,tt9916428,The Secret of China,"Adventure,History,War, Adventure,History,War, ...","nm0422639, nm0910951, nm3370295, nm3611859, nm...","actress, director, actor, actor, actor, actor,...","Wenli Jiang, Jixing Wang, Shenyang Xiao, Kenan...","The Secret of China Adventure,History,War, Adv..."
289030,tt9916538,Kuambil Lagi Hatiku,"Drama, Drama, Drama, Drama, Drama","nm10041459, nm1266058, nm1417182, nm4457074, n...","actor, actress, actress, director, actress","Sahil Shah, Ria Irawan, Cut Mini Theo, Azhar K...","Kuambil Lagi Hatiku Drama, Drama, Drama, Drama..."


In [88]:
aggregated_df=aggregated_df.replace(r'\s*\\N\s*', '', regex=True)

In [89]:
aggregated_df

,tconst,primaryTitle,genres,nconst,category,primaryName,features
0,tt0000009,Miss Jerry,"Romance, Romance, Romance, Romance","nm0063086, nm0085156, nm0183823, nm1309758","actress, director, actor, actor","Blanche Bayliss, Alexander Black, William Cour...","Miss Jerry Romance, Romance, Romance, Romance ..."
1,tt0000147,The Corbett-Fitzsimmons Fight,"Documentary,News,Sport",nm0714557,director,Enoch J. Rector,"The Corbett-Fitzsimmons Fight Documentary,News..."
2,tt0000502,Bohemios,",,","nm0063413, nm0215752, nm0252720","director, actor, actor","Ricardo de Baños, Antonio del Pozo, El Mochuelo","Bohemios,,nm0063413, nm0215752, nm0252720 dire..."
3,tt0000574,The Story of the Kelly Gang,"Action,Adventure,Biography, Action,Adventure,B...","nm0846879, nm0846887, nm0846894, nm1431224, nm...","director, actress, actor, actor, actor","Charles Tait, Elizabeth Tait, John Tait, Nicho...","The Story of the Kelly Gang Action,Adventure,B..."
4,tt0000591,The Prodigal Son,"Drama, Drama, Drama, Drama, Drama","nm0141150, nm0332182, nm0906197, nm1323543, nm...","director, actor, actor, actress, actress","Michel Carré, Henri Gouget, Georges Wague, Chr...","The Prodigal Son Drama, Drama, Drama, Drama, D..."
...,...,...,...,...,...,...,...
289027,tt9916270,Il talento del calabrone,"Thriller, Thriller, Thriller, Thriller, Thriller","nm0144812, nm1480867, nm2063290, nm3080119, nm...","actor, director, actress, actor, actress","Sergio Castellitto, Giacomo Cimini, Anna Fogli...","Il talento del calabrone Thriller, Thriller, T..."
289028,tt9916362,Coven,"Drama,History, Drama,History, Drama,History, D...","nm0107165, nm0266723, nm10678594, nm1893148, n...","actor, actor, actress, director, actress","Alex Brendemühl, Daniel Fanego, Garazi Urkola,...","Coven Drama,History, Drama,History, Drama,Hist..."
289029,tt9916428,The Secret of China,"Adventure,History,War, Adventure,History,War, ...","nm0422639, nm0910951, nm3370295, nm3611859, nm...","actress, director, actor, actor, actor, actor,...","Wenli Jiang, Jixing Wang, Shenyang Xiao, Kenan...","The Secret of China Adventure,History,War, Adv..."
289030,tt9916538,Kuambil Lagi Hatiku,"Drama, Drama, Drama, Drama, Drama","nm10041459, nm1266058, nm1417182, nm4457074, n...","actor, actress, actress, director, actress","Sahil Shah, Ria Irawan, Cut Mini Theo, Azhar K...","Kuambil Lagi Hatiku Drama, Drama, Drama, Drama..."


In [93]:
aggregated_df.shape

(289032, 7)

In [94]:
vectorizer = CountVectorizer()

In [95]:
feature_matrix = vectorizer.fit_transform(aggregated_df['features'].astype(str))

In [96]:
feature_df = pd.DataFrame.sparse.from_spmatrix(feature_matrix, index=aggregated_df.index.tolist())

In [97]:
feature_df

,0,1,2,3,4,5,6,7,8,9,...,856849,856850,856851,856852,856853,856854,856855,856856,856857,856858
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289029,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
svd = TruncatedSVD(n_components=35)

In [99]:
latent_df = svd.fit_transform(feature_df)

In [100]:
latent_df.shape

(289032, 35)

In [101]:
latent_df

array([[ 2.14045773e+00,  7.53740419e-01, -3.16403360e-01, ...,
        -3.45659259e-02, -2.60682545e-03, -6.77586696e-03],
       [ 2.54954220e-01,  7.66124885e-02,  1.45593467e-01, ...,
         1.28481265e-02, -2.08689969e-02, -3.17476858e-02],
       [ 1.23745201e+00,  3.98299523e-01,  7.16530849e-01, ...,
         7.40893164e-01,  9.45316466e-02,  1.94544087e-01],
       ...,
       [ 3.89303524e+00,  1.19782826e+00,  3.15829616e+00, ...,
        -9.19056021e-03, -3.87024996e-02, -1.61264342e-01],
       [ 4.95025390e+00, -2.38689950e+00, -1.05619937e+00, ...,
        -1.56140865e-02, -9.56101354e-03, -4.23173101e-02],
       [ 2.24906670e+00,  7.27293973e-01,  1.35446452e+00, ...,
        -3.47942553e-02, -3.03191983e-02, -6.77002311e-02]])

In [102]:
latent_df = pd.DataFrame(latent_df[:,0:35], index=aggregated_df.primaryTitle.tolist())

In [103]:
latent_df

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
Miss Jerry,2.140458,0.753740,-0.316403,1.140792,3.705241,0.094924,-0.901395,-0.136037,-0.509551,0.078592,...,-0.091637,0.014365,0.022907,-0.050960,-0.043584,-0.045177,-0.000947,-0.034566,-0.002607,-0.006776
The Corbett-Fitzsimmons Fight,0.254954,0.076612,0.145593,0.115201,-0.001288,-0.185223,-0.361216,-0.075550,-0.126625,0.222877,...,0.190316,-0.063267,-0.247699,-0.164605,0.040340,-0.048677,-0.041668,0.012848,-0.020869,-0.031748
Bohemios,1.237452,0.398300,0.716531,0.186608,0.035330,-0.414820,-0.969335,-0.441239,-0.860878,-0.049858,...,-0.139771,-0.027461,-0.645066,0.874091,0.049106,-0.011565,-0.250064,0.740893,0.094532,0.194544
The Story of the Kelly Gang,3.178840,1.224723,4.744671,-3.328842,1.830449,-2.839065,-0.439225,-0.495866,1.732488,-0.932279,...,1.738184,-0.008499,0.313249,0.181377,-0.727351,0.370023,-0.369731,0.035911,-0.092720,-0.027182
The Prodigal Son,5.201673,-2.286664,-0.733432,0.034174,-0.734619,-0.470876,0.181775,0.128944,0.210252,0.258218,...,0.710578,-0.088915,-0.229806,-0.177644,0.045717,-0.072811,-0.038519,-0.067436,-0.046201,0.080104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Il talento del calabrone,2.248937,0.528788,2.472102,2.727699,-0.116574,1.107553,1.067437,-3.221885,0.803573,0.390607,...,-0.132660,0.009981,0.003806,0.000891,0.064184,-0.040815,-0.064952,-0.004283,-0.021195,-0.052901
Coven,5.281953,-2.383848,-0.724928,-0.023597,-0.742742,-0.660294,-0.036461,-0.029818,-0.041882,0.227715,...,-0.290307,-0.006196,-0.002785,-0.069965,-0.002197,-0.053883,0.002548,-0.027977,-0.009207,-0.021697
The Secret of China,3.893035,1.197828,3.158296,-1.041474,0.891223,-3.908919,-4.682526,-2.223179,2.233502,-1.528918,...,0.359843,-0.021759,0.514301,0.319023,0.203806,-0.052569,-0.055399,-0.009191,-0.038702,-0.161264
Kuambil Lagi Hatiku,4.950254,-2.386899,-1.056199,0.170818,-0.714571,-0.320177,0.801463,0.541066,1.064988,0.643749,...,-0.117519,-0.030172,0.035567,-0.073915,-0.004833,-0.046521,-0.012404,-0.015614,-0.009561,-0.042317


In [104]:
# Connect to the SQLite database
conn = sqlite3.connect('latent_features.db')

In [105]:
# Store the DataFrame in the database
latent_df.to_sql('latent_features', conn, if_exists='replace', index=True)

289032

In [106]:
# Close the database connection
conn.close()

In [107]:
# Connect to the SQLite database
conn = sqlite3.connect('latent_features.db')

In [108]:
# Load the DataFrame from the database
latent_df = pd.read_sql_query('SELECT * FROM latent_features', conn,index_col='index')

In [109]:
# Close the database connection
conn.close()

In [110]:
latent_df

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
index,,,,,,,,,,,,,,,,,,,,,
Miss Jerry,2.140458,0.753740,-0.316403,1.140792,3.705241,0.094924,-0.901395,-0.136037,-0.509551,0.078592,...,-0.091637,0.014365,0.022907,-0.050960,-0.043584,-0.045177,-0.000947,-0.034566,-0.002607,-0.006776
The Corbett-Fitzsimmons Fight,0.254954,0.076612,0.145593,0.115201,-0.001288,-0.185223,-0.361216,-0.075550,-0.126625,0.222877,...,0.190316,-0.063267,-0.247699,-0.164605,0.040340,-0.048677,-0.041668,0.012848,-0.020869,-0.031748
Bohemios,1.237452,0.398300,0.716531,0.186608,0.035330,-0.414820,-0.969335,-0.441239,-0.860878,-0.049858,...,-0.139771,-0.027461,-0.645066,0.874091,0.049106,-0.011565,-0.250064,0.740893,0.094532,0.194544
The Story of the Kelly Gang,3.178840,1.224723,4.744671,-3.328842,1.830449,-2.839065,-0.439225,-0.495866,1.732488,-0.932279,...,1.738184,-0.008499,0.313249,0.181377,-0.727351,0.370023,-0.369731,0.035911,-0.092720,-0.027182
The Prodigal Son,5.201673,-2.286664,-0.733432,0.034174,-0.734619,-0.470876,0.181775,0.128944,0.210252,0.258218,...,0.710578,-0.088915,-0.229806,-0.177644,0.045717,-0.072811,-0.038519,-0.067436,-0.046201,0.080104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Il talento del calabrone,2.248937,0.528788,2.472102,2.727699,-0.116574,1.107553,1.067437,-3.221885,0.803573,0.390607,...,-0.132660,0.009981,0.003806,0.000891,0.064184,-0.040815,-0.064952,-0.004283,-0.021195,-0.052901
Coven,5.281953,-2.383848,-0.724928,-0.023597,-0.742742,-0.660294,-0.036461,-0.029818,-0.041882,0.227715,...,-0.290307,-0.006196,-0.002785,-0.069965,-0.002197,-0.053883,0.002548,-0.027977,-0.009207,-0.021697
The Secret of China,3.893035,1.197828,3.158296,-1.041474,0.891223,-3.908919,-4.682526,-2.223179,2.233502,-1.528918,...,0.359843,-0.021759,0.514301,0.319023,0.203806,-0.052569,-0.055399,-0.009191,-0.038702,-0.161264


In [111]:
# Get the latent feature representation for "The Conjuring"
conjuring_latent = latent_df.loc[str("Coven")].values.reshape(1, -1)

In [112]:
movie_name = "The Conjuring"
if movie_name in latent_df.index:
    conjuring_latent = latent_df.loc[movie_name].values.reshape(1, -1)
    # Rest of your code to compute movie recommendations using conjuring_latent
else:
    print("Movie not found in the latent features DataFrame.")

In [113]:
conjuring_latent

array([[ 2.80664123e+00,  8.77142076e-01,  4.27117452e+00,
         6.27349918e+00, -7.65220467e-01,  4.85889516e-01,
         1.53135826e+00,  4.22866951e-04,  9.51438410e-01,
        -1.40210308e+00,  3.62776114e-01, -1.29437668e-01,
         3.00210083e+00,  1.58547205e+00, -7.03903941e-01,
        -9.70238994e-02,  1.37210443e-01, -1.41759279e-02,
         2.66484945e-01,  1.97351599e-01, -5.82335639e-02,
         2.63667269e-01,  1.00522201e-01, -1.91813844e-01,
         3.47321176e-01,  5.07665128e-01, -1.41661135e-02,
        -2.18098630e-01, -1.88684324e-01, -5.30499042e-03,
        -7.97605434e-02,  3.36864631e-02, -6.37934619e-02,
        -1.28118126e-02, -4.01327729e-02]])

In [114]:
from sklearn.metrics.pairwise import cosine_similarity
# Compute cosine similarity with other movies
similarity_scores = cosine_similarity(conjuring_latent, latent_df.values)



In [115]:
# Get the top-N similar movies
N = 15  # Number of recommended movies
similar_movies_indices = similarity_scores.argsort()[0][-N-1:-1][::-1]  # Exclude the movie itself

In [116]:
recommended_movies = latent_df.iloc[similar_movies_indices].index.tolist()

print("Recommended movies for 'The Conjuring':")
for movie in recommended_movies:
    print(movie)

Recommended movies for 'The Conjuring':
The Night Has Eyes
The Pyramid
Through the Devil's Eyes
The Room Next Door
The Children
The Vigil
The Gruesome Twosome
The Levenger Tapes
Don't Go to the Reunion
The Eighteenth Angel
The Ghost Club: Spirits Never Die
The Strangers
The Grudge
The Visit
The Moretti House
